# Process Media

In [ ]:
import json
import os

import pandas as pd
from tqdm import tqdm

In [ ]:
os.chdir("../../data/raw_data")

In [ ]:
media = ""

In [ ]:
medium = pd.read_csv(f"{media}.csv")
medium['tags'] = '[]'
medium['links'] = '[]'

In [ ]:
# Imports anidb tags from https://github.com/manami-project/anime-offline-database
if media == "anime":
    try:
        if os.path.exists('anime-offline-database.json'):
            os.remove('anime-offline-database.json')
        json_fn = "https://github.com/manami-project/anime-offline-database/raw/master/anime-offline-database.json"
        ! wget $json_fn 
        with open('anime-offline-database.json', 'r') as f:
            db = json.load(f)    
        for entry in tqdm(db['data']):
            mal_id = [x for x in entry['sources'] if 'myanimelist' in x]
            if mal_id:
                mal_id = int(mal_id[0].split('/')[-1])
                medium.loc[lambda x: x.anime_id == mal_id, 'tags'] = str(entry['tags'])
                medium.loc[lambda x: x.anime_id  == mal_id, 'links'] = str(entry['sources'])        
    except Exception as e:
        print(str(e))

In [ ]:
for i in range(len(medium)):
    if medium.loc[i, "links"] == '[]':
        uid = medium.loc[i, f"{media}_id"]
        medium.loc[i, "links"] = f"['https://myanimelist.net/{media}/{uid}']"

In [ ]:
os.makedirs("../processed_data", exist_ok=True)

In [ ]:
medium.to_csv(f'../processed_data/{media}.csv', index=False)